### A.1 Pytorch
1. Tensor library: torch.tensor()
2. Automatic Differentiation engine:
    - import torch.nn.functional as F
    - from torch.autograd import grad
3. Deep learning library: torch.nn.Module  

In [1]:
import torch

In [2]:
torch.__version__

'2.7.1'

In [3]:
torch.cuda.is_available()

False

In [5]:
torch.backends.mps.is_available()

True

### A.2 Understanding Tensors

In [6]:
import numpy as np

In [9]:
# 1) Scalars, Vectors, Matrices, and Tensors

tensor0d = torch.tensor(1) # scalar
tensor1d = torch.tensor([1,2,3]) # vector
tensor2d = torch.tensor([[1,2], [3, 4]]) # matrix

# Using a nested Python list
tensor3d_1 = torch.tensor([[[1,2], [3,4]],
                           [[5,6], [7,8]]])
# Using Numpy Array
aray3d = np.array([[[1,2], [3,4]],
                    [[5,6], [7,8]]])
tensor3d_2 = torch.tensor(aray3d) # copies
tensor3d_3 = torch.from_numpy(aray3d) # shares memory with NumPy array
# if aray3d has been changed, tensor3d_3 would be changed as well

In [10]:
# 2) Tensor data types

tensor1d = torch.tensor([1,2,3])
print(tensor1d.dtype)

floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

floatvec = tensor1d.to(torch.float32)
print(floatvec)
print(floatvec.dtype)

torch.int64
torch.float32
tensor([1., 2., 3.])
torch.float32


In [12]:
# 3) Tensor operation

tensor2d = torch.tensor([[1,2,3],
                         [4,5,6]])
print(f"Original Tensor: {tensor2d}")
print(f"Original Tensor shape: {tensor2d.shape}")

print(f"Reshape: {tensor2d.reshape(3,2)}")
print(f"View: {tensor2d.view(3,2)}")
print(f"Shape of reshape and view: {tensor2d.reshape(3,2).shape}")

print(f"Transpose: {tensor2d.T}")
print(f"Shape of transpose: {tensor2d.T.shape}")

print(f"Matmul: {tensor2d.matmul(tensor2d.T)}")
print(f"@: {tensor2d @ tensor2d.T}")
print(f"Shape of matmul: {tensor2d.matmul(tensor2d.T).shape}")

Original Tensor: tensor([[1, 2, 3],
        [4, 5, 6]])
Original Tensor shape: torch.Size([2, 3])
Reshape: tensor([[1, 2],
        [3, 4],
        [5, 6]])
View: tensor([[1, 2],
        [3, 4],
        [5, 6]])
Shape of reshape and view: torch.Size([3, 2])
Transpose: tensor([[1, 4],
        [2, 5],
        [3, 6]])
Shape of transpose: torch.Size([3, 2])
Matmul: tensor([[14, 32],
        [32, 77]])
@: tensor([[14, 32],
        [32, 77]])
Shape of matmul: torch.Size([2, 2])


### A.3 Seeing models as computation graphs

In [13]:
import torch.nn.functional as F

y = torch.tensor([1.0]) # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight params
b = torch.tensor([0.0]) # bias

z = x1 * w1 + b # input * weight + bias
a = torch.sigmoid(z) # activation
loss = F.binary_cross_entropy(a, y) # loss(predict, true)
print(loss)

tensor(0.0852)


### A.4 Automatic differentiation made easy
- option: requires_grad=True

In [15]:
from torch.autograd import grad

y = torch.tensor([1.0]) # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2], requires_grad=True) # weight params
b = torch.tensor([0.0], requires_grad=True) # bias

z = x1*w1+b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

loss.backward()
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898])
tensor([-0.0817])


### A.5 Implementing multilayer neural networks

In [16]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [17]:
model = NeuralNetwork(50, 3) #input:30, output:3
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [18]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

2213


In [19]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.0980,  0.1206,  0.0794,  ..., -0.0320, -0.0515, -0.0909],
        [-0.0786,  0.0261, -0.0096,  ...,  0.0287,  0.1168,  0.1230],
        [ 0.1335,  0.0078,  0.1390,  ...,  0.0622, -0.0184, -0.1265],
        ...,
        [-0.0206, -0.1390, -0.0188,  ..., -0.1378,  0.1035, -0.0060],
        [ 0.0005, -0.0458,  0.1236,  ..., -0.0569, -0.1321, -0.0212],
        [-0.0277,  0.0020, -0.0279,  ...,  0.0635,  0.0442,  0.0301]],
       requires_grad=True)


In [20]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [21]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [25]:
torch.manual_seed(123)

X = torch.rand((1,50))
out = model(X) # forward pass
print(out)
print(out.shape)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)
torch.Size([1, 3])


AddmmBackward0: Matrix Multiplication (mm) followed by an addition (Add)

In [26]:
# inference (don't need to use gradient like training process)
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [27]:
# non-linear activation func in output layer
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


### A.6 Setting up efficient data loaders
from torch.utils.data import Dataset

In [28]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

In [29]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [32]:
from torch.utils.data import Dataset, DataLoader

# Generate Dataset 
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    def __len__(self):
        return self.labels.shape[0] # row

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

# Load Dataset based on batch size and shuffle
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

5


In [33]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [34]:
# drop_last
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [35]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


### A.7 A typical training loop

In [37]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train/Val loss: {loss:.2f}")

    model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val loss: 0.00


In [38]:
model.eval() # evaluation

with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [39]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

predicts = torch.argmax(probas, dim=1)
print(predicts)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])


In [40]:
print(predicts == y_train)
print(torch.sum(predicts==y_train))

tensor([True, True, True, True, True])
tensor(5)


In [43]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predicts = torch.argmax(logits,dim=1)
        compare = labels == predicts
        correct += torch.sum(compare)
        total_examples += len(compare)
    return (correct/total_examples).item()

In [44]:
print(compute_accuracy(model, train_loader))
print(compute_accuracy(model, test_loader))

1.0
1.0


### A.9 Optimizing training performance with GPUs

In [45]:
print(torch.__version__)

2.7.1


In [48]:
print(torch.backends.mps.is_available())

True


In [52]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tensor_1 = torch.tensor([1., 2., 3.]).to(device)
tensor_2 = torch.tensor([4., 5., 6.]).to(device)
print(tensor_1+tensor_2)

tensor([5., 7., 9.], device='mps:0')


In [53]:
# 1. fix the random seed
torch.manual_seed(123)

# 2. Load Model
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# 3. fix the device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# 4. Load optimizer
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

# 5. train 
num_epochs = 3
for epoch in range(num_epochs):
    model.train() # 5.1 model train mode
    for batch_idx, (features, labels) in enumerate(train_loader): # iter epochs
        features,labels = features.to(device), labels.to(device)  # 5.1.1. convert in GPU mode
        logits = model(features) # 5.1.2 forward
        loss = F.cross_entropy(logits, labels) # 5.1.3 calculate loss

        # 5.1.4 backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train/Val loss: {loss:.2f}")
    
    model.eval() # 5.2. evaluation for each epoch

Epoch: 001/003 | Batch 000/002 | Train/Val loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val loss: 0.00
